# Automated Model Evaluation

Now that we have a general model function, we can create a script that makes models out of different combinations of features and displays the best results.

In [76]:
import itertools
from scipy.special import comb
import pandas as pd
import GeneralModel as gm
from tqdm import tqdm_notebook as tqdm

In [2]:
# retrieves data
merged = pd.read_csv('../../DataPlus/feature_dataframe.csv')

In [3]:
model_df = gm.prepare_df(merged, cat_vars=['edu_binary', 'marry_binary', 'Advice1'], cont_vars=[])

# of Data Points: 358


In [5]:
fscore, metrics, auc_score, feat_info = gm.general_model(model_df, algorithm='rf', print_feat=True)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


             Feature    Weight
0  No College Degree  0.193892
1        Not Married  0.112416
2                 AR  0.044520
3                 AS  0.051509
4                ASR  0.280635
5                  R  0.100839
6                  S  0.133658
7                 SR  0.082531


F-score: 0.469
AUC: 0.607


In [123]:
"""
    Given a set of categorical and continuous features, the function will iterate
    over the combinations of features and present the results in a dataframe
    
    Columns in dataframe: features (list), fscore, auc, feature importance (if applicable) 
    
    input:
        - df: original dataframe
        - min_feats: minimum subset size taken from feature list
        - sort: column by which to sort dataframe (by AUC or fscore)
        - algorithm: type of algorithm
        - cat_vars: list of categorical variables
        - cont_vars: list of continuous variables
"""
def model_evaluation(df, min_feats=1, sort=None, algorithm='rf', cat_vars=[], cont_vars=[]):
    total_features = cat_vars + cont_vars;
    
    # keeps track of results
    results = pd.DataFrame();
    
    if min_feats > len(total_features):
        print('error: minimum subset size exceeds feature set')
        return None
    
    # iterates through all subsets greater than min_feats
    for i in range(len(total_features)-min_feats+1):
        subset_size = i + min_feats
        
        num_combinations = int(comb(len(total_features), subset_size))
        print('# of combinations for subset size {}: {}'.format(subset_size, num_combinations))
        for var_set in tqdm(itertools.combinations(total_features, subset_size)):
            # separates the feature subset into categorical and continuous
            cont_var_set = [var for var in var_set if var in cont_vars]
            cat_var_set = [var for var in var_set if var in cat_vars]
            
            # prepares model for feature subset
            model_df = gm.prepare_df(df, cont_vars=cont_var_set, cat_vars=cat_var_set, print_dims=False)
            fscore, _, auc_score, feat_importance_df = gm.general_model(model_df, algorithm=algorithm, print_metrics=False, tqdm_on=False)
        
            feature_string = ', '.join(list(var_set))
            num_feats = subset_size
            
            # handles feature importance field
            ordered_feat_importance = feat_importance_df.sort_values('Weight', ascending=False)
            ordered_weights = ordered_feat_importance['Weight']
            ordered_feats = ordered_feat_importance['Feature']
            
            import_array = []
            for i in range(len(ordered_feat_importance.index)):
                import_array.append('{}: {}'.format(ordered_feats[i], ordered_weights[i]))
            import_string = ', '.join(import_array)
            
            # adds result of model to dataframe
            results = results.append(pd.DataFrame({'features': feature_string, 'fscore': fscore, 
                                                   'auc': auc_score, 'num_features': num_feats, 'feat_importance': import_string}, index=[0]))
    
    # sorts dataframe in descending order of desired metric
    if sort is not None : results = results.sort_values(sort, ascending=False)
        
    # columns in right order
    col_order = [sort, 'features'] + [col for col in results.columns if col not in [sort, 'features']]
    results = results[col_order]
    
    return results

In [124]:
cat_feat_set = ['marry_binary', 'gleason', 'edu_binary']
cont_feat_set = ['age']

In [125]:
result = model_evaluation(merged, min_feats=2, sort='fscore', cat_vars=cat_feat_set, cont_vars=cont_feat_set)

# of combinations for subset size 2: 6


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

# of combinations for subset size 3: 4


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

# of combinations for subset size 4: 1


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [126]:
result

,fscore,features,auc,feat_importance,num_features
0,0.714200,"marry_binary, gleason",0.765810,"Not Married: 0.03515698074864761, 7.0: 0.96484...",2
0,0.713641,"gleason, edu_binary",0.767060,"7.0: 0.9342252557818455, No College Degree: 0....",2
0,0.705883,"marry_binary, gleason, edu_binary",0.758385,"Not Married: 0.03913995449133547, 7.0: 0.88577...",3
0,0.634528,"marry_binary, gleason, age",0.707258,"age: 0.5611225693415214, Not Married: 0.025938...",3
0,0.617423,"gleason, age",0.695929,"age: 0.48647022031092496, 7.0: 0.513529779689075",2
0,0.615006,"marry_binary, gleason, edu_binary, age",0.695933,"age: 0.5027569207492849, Not Married: 0.041251...",4
0,0.612877,"gleason, edu_binary, age",0.688056,"age: 0.5301154529826299, 7.0: 0.42540725250057...",3
0,0.421683,"marry_binary, edu_binary, age",0.564925,"age: 0.9018332672052193, Not Married: 0.035947...",3
0,0.411335,"edu_binary, age",0.563885,"age: 0.9322607619754328, No College Degree: 0....",2
0,0.401394,"marry_binary, age",0.566444,"age: 0.9619505225710205, Not Married: 0.038049...",2


In [127]:
result2 = model_evaluation(merged, algorithm='lr', min_feats=2, sort='fscore', cat_vars=cat_feat_set, cont_vars=cont_feat_set)

# of combinations for subset size 2: 6


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

# of combinations for subset size 3: 4


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

# of combinations for subset size 4: 1


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [128]:
result2

,fscore,features,auc,feat_importance,num_features
0,0.715252,"marry_binary, gleason, edu_binary",0.765929,"Not Married: 0.3233063689907592, 7.0: -2.19513...",3
0,0.714162,"gleason, edu_binary",0.767452,"7.0: -2.2758832305504684, No College Degree: 0...",2
0,0.713834,"marry_binary, gleason",0.765833,"Not Married: 0.4033031488316615, 7.0: -2.35929...",2
0,0.686561,"marry_binary, gleason, age",0.749091,"age: 0.7142567727187229, Not Married: 0.521025...",3
0,0.675332,"marry_binary, gleason, edu_binary, age",0.736286,"age: 0.7085023472310987, Not Married: 0.449471...",4
0,0.671233,"gleason, age",0.734274,"age: 0.5615745136030384, 7.0: -2.4499931567908733",2
0,0.663497,"gleason, edu_binary, age",0.729679,"age: 0.6650150925791773, 7.0: -2.4086887905512...",3
0,0.274266,"edu_binary, age",0.550508,"age: 0.4608322047864485, No College Degree: 0....",2
0,0.257041,"marry_binary, edu_binary, age",0.535992,"age: 0.4241326733270607, Not Married: 0.167281...",3
0,0.175061,"marry_binary, age",0.514135,"age: 0.37296155514055745, Not Married: 0.40515...",2


In [133]:
result2.iloc[0]['feat_importance']

'Not Married: 0.3233063689907592, 7.0: -2.195132471760106, No College Degree: 0.41054528017780667'

In [148]:
model_df = gm.prepare_df(merged, cat_vars=['gleason', 'edu_binary', 'marry_binary'], cont_vars=[])

# of Data Points: 390


In [149]:
gm.general_model(model_df, algorithm='lr')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


F-score: 0.716
AUC: 0.766


(0.7158392319236815,
 {'negative precision': 0.8681922139309068,
  'negative recall': 0.7180555555555556,
  'positive precision': 0.6388248546010361,
  'positive recall': 0.8139682539682541},
 0.7660119047619048,
              Feature    Weight
 0                  7 -2.158512
 1  No College Degree  0.432922
 2        Not Married  0.335120)